# 데이터 수집 
## 데이터 저장에 필요한 데이터베이스 생성(SQLite)
**데이터베이스명** : TeamProjectFinal

In [1]:
import pandas as pd 
import numpy as np  
from pyfile.connect2db import * # 데이터베이스 연결
conn,cursor = create_con()
# DB에 있는 테이블 목록 확인하기
pd.read_sql('select tbl_name from sqlite_master;',conn)

,tbl_name
0,cafe_post_list_naver_test
1,cafe_post_comment_naver
2,cafe_post_contents_naver
3,cafe_post_list_naver
4,comments_hy_kona_naver
5,commnets_kia_stonic_naver
6,commnets_sam_qm3_naver
7,commnets_ssy_tivoli_naver
8,kona
9,kona_total


### 카페 게시글 수집에 필요한 데이터 생성 

In [2]:
# create_post_list_table(conn,cursor)

## 자동차 판매량 데이터 수집
공공 데이터 포털에서 코나, 티볼리, 스토닉, QM3의 데이터를 불러오기 

In [3]:
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote
from xml.etree import ElementTree

decode_key = unquote('xc4D5ifBCowmhcub66MRZfZGkwDJ05AUj4Quu5kV7in%2Fp8HB5CAUFNbvvxtCf2llE5NPDuI7x4P4vEeVNbw1OA%3D%3D')

url = 'http://apis.data.go.kr/B553881/newRegistInfoService/newRegistInfoService'
queryParams = '?' + urlencode({ 
quote_plus('ServiceKey') : decode_key, 
                               quote_plus('ServiceKey') : decode_key, 
                               quote_plus('registYy') : '2019',     # 등록년
                               quote_plus('registMt') : '05',       # 등록월
                               quote_plus('vhctyAsortCode') : '1',  # 차종코드 (1.경형)
                               quote_plus('registGrcCode') : '2',   # 등록지역코드 (1.서울 / 2.부산 / 3.대구 / 4.인천 / 5.광주 / 6.대전 / 7.울산 / 8.세종 / 9.경기 / 10.강원 / 11.충북 / 12.충남 / 13.전북 / 14.전남 / 15.경북 / 16.경남 / 17.제주)
                               quote_plus('useFuelCode') : '9',     # 연료코드 (2.경유 / 6.전기 / 8.하이브리드(휘발유+전기) / 9.휘발유
                               quote_plus('cnmCode') : '006987',    # 차명코드
                               quote_plus('prposSeNm') : '1',       # 용도구분 (1.자가용)
                               quote_plus('sexdstn') : '여자',      # 성별 (남자 / 여자)
                               quote_plus('agrde') : '4',           # 연령 (1.10대 / 2.20대 / 3.30대 / 4.40대 / 5.50대 / 6.60대 /7.70대 / 8.80대)
                               quote_plus('dsplvlCode') : '4',      # 배기량코드 (1.800cc미만 / 2.1000cc미만 / 3.1500cc미만 / 4.2000cc미만)
                               quote_plus('hmmdImpSeNm') : '국산',  # 국가 (국산)
                               quote_plus('prye') : '2019' })       # 모델년도

request = Request(url + queryParams)
request.get_method = lambda: 'GET'
response_body = urlopen(request).read().decode('utf-8')

root = ElementTree.fromstring(response_body)
total = root.find("body").find("dtaCo")

print('XML을 파싱 후 2019년 5월 부산(2) 40대(4) 여성이 등록한 코나(006987) 대수 {}건의 정보를 가져왔습니다.'.format(total.text))

XML을 파싱 후 2019년 5월 부산(2) 40대(4) 여성이 등록한 코나(006987) 대수 8건의 정보를 가져왔습니다.


## 네이버 뉴스 스크래핑

In [4]:
from pyfile.news_scraping import * 
import os 

In [5]:
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''
< naver 뉴스 검색시 리스트 크롤링하는 프로그램 > _select사용
- 크롤링 해오는 것 : 링크,제목,신문사,날짜,내용요약본
- 날짜,내용요약본  -> 정제 작업 필요
- 리스트 -> 딕셔너리 -> df -> 엑셀로 저장 
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''
main()
print('스크래핑 된 파일 목록입니다')
os.listdir('./data/news/')

입력 형식에 맞게 입력해주세요.
시작하시려면 Enter를 눌러주세요.
최대 크롤링할 페이지 수 입력하시오: 1
검색어 입력: 베뉴
뉴스 검색 방식 입력(관련도순=0, 최신순=1, 오래된순=2): 1
시작날짜 입력(YYYY.MM.DD): 2019.07.01
끝날짜 입력(YYYY.MM.DD): 2019.07.02
1
크롤링이 완료되었습니다.
스크래핑 된 파일 목록입니다


['2019-7-19  13시 0분 56초 merging.xlsx']

## 네이버 카페 게시글 및 댓글 스크래핑 
게시글 10개 페이지만 스크래핑 해서 cafe_post_list_naver_test에 저장 

In [6]:
from pyfile.cafe_scraping import * 
from selenium import webdriver
from bs4 import BeautifulSoup
from urllib.request import urlopen
import time 

# 네이버 로그인 
driver = login_to_naver('mindal0115','nalzoest1807')

# 오픈할 카페와 게시판 지정 
cafe_url = 'https://cafe.naver.com/jsmusic'
menu = '//*[@id="menuLink215"]'
get_to_cafe(driver,cafe_url,menu)

# 게시글 목록 수집
# scrap_post(driver,conn,cursor)

In [7]:
df = pd.read_sql('select * from cafe_post_list_naver_test;',conn,parse_dates=['p_date'])
df.iloc[0:5,:]

,board_number,article,p_nick,p_date,p_view
0,1035033,제차만 그럴까요??ㅠㅠ,경남ll꼬꼬댁,2019-07-05,80
1,1034978,18인치 순정 휠타이어 구매하고싶습니다~,서울ll히츠블루,2019-07-05,50
2,1034948,코나 안전벨트 소리 안나게할수있을까요?,경기고양ll호나,2019-07-18,113
3,1034941,출고한지 한달도 안됐는데 브레이크..,경기화성llk532,2019-07-18,139
4,1034936,기름 만땅시 몇일 탈수 있나요?,아산ll동동,2019-07-18,191
